In [4]:
!pip install lightgbm

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import numpy as np
import pandas as pd



In [8]:

#2. 데이터 불러오기
import pandas as pd

# 데이터 경로에서 파일 불러오기
train = pd.read_csv('./data/train.csv', encoding='utf-8')
test = pd.read_csv('./data/test.csv', encoding='utf-8')
sample_submission = pd.read_csv('./data/sample_submission.csv', encoding='utf-8')

# train 데이터 확인
print("=== [Train 데이터 미리보기] ===")
display(train.head())  # 주피터에서는 display()가 더 예쁨

print("\n=== [Train 데이터 정보] ===")
train.info()


=== [Train 데이터 미리보기] ===


,ID,generation,school1,major type,major1_1,major1_2,major_data,job,class1,class2,...,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,incumbents_lecture_scale_reason,interested_company,expected_domain,contest_participitation,idea_contest,onedayclass_topic,withdrawal
0,TRAIN_0000,8,3e0057be3ecde495d1d6aab0bdc3092cce94440a6a4f68...,단일 전공,IT(컴퓨터 공학 포함),NaN,False,대학생,4,NaN,...,"국내 대기업 IT 계열 (금융, 제조 ...)","온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,인원이 너무 많지 않아야 편하게 들을 수 있어서,삼성이나 하이닉스,AI,NaN,NaN,C++ 응용,1
1,TRAIN_0001,8,c7c76d1a4c9d719d17c899a3ca9de153452a22163fe083...,"복수 전공 ( 다중전공, 이중전공 포함 )",IT(컴퓨터 공학 포함),IT(컴퓨터 공학 포함),False,대학생,4,NaN,...,국내 빅테크 IT 계열 (네카쿠라배당토),"온,오프라인 동시",10명 내외의 강의 리스너와 1명의 현직자,인원이 적어야 집중할 수 있고 질문 하기 쉽다.,네이버,AI,NaN,NaN,.,1
2,TRAIN_0002,8,1ed9296bd2a05ea36fabb5171a003b7320bb21a53d6135...,단일 전공,IT(컴퓨터 공학 포함),NaN,True,대학생,4,NaN,...,"국내 대기업 IT 계열 (금융, 제조 ...)","온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,너무 소수인원은 조금 부담스럽고 너무 많은 인원은 집중도가 떨어질거같아서,스포츠분석 기업,AI,139ac02d03d83f839197195e65bc1810ef5282ade32d80...,NaN,C언어 기초,1
3,TRAIN_0003,8,5ec50ef5a0c0416652f8d67a89ef4c02eb0c6a61b5d0a8...,단일 전공,IT(컴퓨터 공학 포함),NaN,True,대학생,7,NaN,...,국내 빅테크 IT 계열 (네카쿠라배당토),"온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,너무 많은 인원이 있을 경우 집중도가 떨어질 거 같습니다.,토스,AI,NaN,NaN,현직자의 취업과정,0
4,TRAIN_0004,8,c7c76d1a4c9d719d17c899a3ca9de153452a22163fe083...,"복수 전공 ( 다중전공, 이중전공 포함 )",IT(컴퓨터 공학 포함),IT(컴퓨터 공학 포함),False,대학생,7,NaN,...,"국내 대기업 IT 계열 (금융, 제조 ...)",오프라인,100명 이상의 리스너와 10명 이상의 현직자,많은 현직자분들의 이야기를 듣고싶어서,네이버 카카오,금융,NaN,NaN,C언어,1



=== [Train 데이터 정보] ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               1056 non-null   object 
 1   generation                       1056 non-null   int64  
 2   school1                          1056 non-null   object 
 3   major type                       1049 non-null   object 
 4   major1_1                         1051 non-null   object 
 5   major1_2                         493 non-null    object 
 6   major_data                       1056 non-null   bool   
 7   job                              1056 non-null   object 
 8   class1                           1056 non-null   int64  
 9   class2                           101 non-null    float64
 10  class3                           2 non-null      float64
 11  class4                           1 non-null      float64
 

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import numpy as np

In [10]:
# 타깃 컬럼명
TARGET = "withdrawal"
ID_COL = "ID"

In [11]:

# 타깃 분리
y = train[TARGET]
X = train.drop(columns=[TARGET])

In [12]:
# train과 test 합치기
all_data = pd.concat([X, test], axis=0).reset_index(drop=True)

In [13]:

# ID 컬럼 제거
all_data = all_data.drop(columns=[ID_COL])

In [14]:
# 범주형 라벨 인코딩
cat_cols = all_data.select_dtypes(include=["object", "bool"]).columns
for col in cat_cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col].astype(str))

In [15]:
# 다시 분할
X_train = all_data[:len(train)]
X_test = all_data[len(train):]

In [16]:
# 예측값 초기화
test_preds = np.zeros(len(X_test))

In [18]:
# Stratified K-Fold 교차 검증
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y)):
    print(f"--- Fold {fold+1} ---")
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dval = lgb.Dataset(X_val, label=y_val)

    params = {
        'objective': 'binary',
        'metric': 'binary_error',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'verbosity': -1,
        'random_state': 42
    }
    from lightgbm import early_stopping, log_evaluation

model = lgb.train(
    params,
    dtrain,
    valid_sets=[dval],
    num_boost_round=1000,
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(100)
    ]
)



--- Fold 1 ---
--- Fold 2 ---
--- Fold 3 ---
--- Fold 4 ---
--- Fold 5 ---
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[19]	valid_0's binary_error: 0.2891


In [21]:
# 테스트 예측 누적 평균
test_preds += model.predict(X_test, num_iteration=model.best_iteration) / skf.n_splits



In [22]:
# 예측 이진화
sample_submission[TARGET] = (test_preds > 0.5).astype(int)


In [23]:

# 저장
sample_submission.to_csv("submit.csv", index=False)
print(" 제출 파일 저장 완료: submit.csv")


 제출 파일 저장 완료: submit.csv
